In [ ]:
%matplotlib inline

import os
import time

%cd F:/Users/Jennifer/onnx/tensorflow-yolov4-tflite
HOME = os.getcwd()
MODEL = "model"
os.environ['PYTHONPATH'] = os.path.join(HOME, "tpu")
os.environ['MODEL'] = MODEL
os.environ['CUDA_VISIBLE_DEVICES'] = ""

In [2]:
!saved_model_cli show --dir ./checkpoints/yolov4.tf --tag_set serve  --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['input_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 416, 416, 3)
      name: serving_default_input_1:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['tf_op_layer_concat_10'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, -1, -1, 3, 85)
      name: StatefulPartitionedCall:0
  outputs['tf_op_layer_concat_11'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, -1, -1, 3, 85)
      name: StatefulPartitionedCall:1
  outputs['tf_op_layer_concat_12'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, -1, -1, 3, 85)
      name: StatefulPartitionedCall:2
Method name is: tensorflow/serving/predict


In [122]:
%matplotlib inline

import numpy as np
import math
import matplotlib.pyplot as plt
import onnxruntime as rt
import cv2
import json
import core.utils as utils

In [123]:
!wget -O img.png -q http://images.cocodataset.org/val2017/000000088462.jpg

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = F:\Program Files (x86)\GnuWin32/etc/wgetrc


In [171]:
input_size = 416

original_image = cv2.imread("img.png")
original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
original_image_size = original_image.shape[:2]

image_data = utils.image_preprocess(np.copy(original_image), [input_size, input_size])
image_data = image_data[np.newaxis, ...].astype(np.float32)

In [172]:
import onnxruntime as rt
sess = rt.InferenceSession("model.onnx")

In [173]:
output_name = sess.get_outputs()[0].name
input_name = sess.get_inputs()[0].name

detections = sess.run([output_name], {input_name: image_data})[0]

In [175]:
def postprocess_bbbox(pred_bbox, ANCHORS, STRIDES, XYSCALE=[1,1,1]):
    for i, pred in enumerate(pred_bbox):
        conv_shape = pred.shape
        output_size = conv_shape[1]
        conv_raw_dxdy = pred[:, :, :, :, 0:2]
        conv_raw_dwdh = pred[:, :, :, :, 2:4]
        xy_grid = np.meshgrid(np.arange(output_size), np.arange(output_size))
        xy_grid = np.expand_dims(np.stack(xy_grid, axis=-1), axis=2)  # [gx, gy, 1, 2]

        xy_grid = np.tile(tf.expand_dims(xy_grid, axis=0), [1, 1, 1, 3, 1])
        xy_grid = xy_grid.astype(np.float)

        # pred_xy = (tf.sigmoid(conv_raw_dxdy) + xy_grid) * STRIDES[i]
        pred_xy = ((tf.sigmoid(conv_raw_dxdy) * XYSCALE[i]) - 0.5 * (XYSCALE[i] - 1) + xy_grid) * STRIDES[i]
        # pred_wh = (tf.exp(conv_raw_dwdh) * ANCHORS[i]) * STRIDES[i]
        pred_wh = (tf.exp(conv_raw_dwdh) * ANCHORS[i])
        pred[:, :, :, :, 0:4] = tf.concat([pred_xy, pred_wh], axis=-1)

    pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_bbox]
    pred_bbox = tf.concat(pred_bbox, axis=0)
    return pred_bbox

In [182]:
import tensorflow as tf
from core.config import cfg

ANCHORS = utils.get_anchors(cfg.YOLO.ANCHORS)
STRIDES = np.array(cfg.YOLO.STRIDES)
XYSCALE = cfg.YOLO.XYSCALE
pred_bbox = postprocess_bbbox(sess, ANCHORS, STRIDES, XYSCALE)

TypeError: 'InferenceSession' object is not iterable